In [1]:
import firebase_admin
from firebase_admin import credentials, db
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
from datetime import datetime

# ================= CONFIGURASI =================
# 1. Lokasi & Waktu
LAT = -7.66  # Kebumen
LON = 109.65
# Format: YYYY-MM-DD
TARGET_DATE = "2025-11-29" 
START_TIME_WIB = "03:09:31"
END_TIME_WIB   = "07:46:31"
FORMAT_KEY = 'UNIX'  # <--- GANTI INI SESUAI DATABASE MU! (Saran: Coba UNIX dulu)

In [2]:
# 2. Firebase Config
# Ganti dengan path file JSON Anda
CRED_PATH = "/work/staklimjerukagung-firebase-admin.json" 
DB_URL = "https://staklimjerukagung-default-rtdb.asia-southeast1.firebasedatabase.app/"
# Path di database tempat data sensor disimpan
# Misal: sensors/kbm-05/logs
DB_NODE_PATH = "/auto_weather_stat/id-05/data" 

# ================= 1. SETUP KONEKSI =================
# Setup Firebase
if not firebase_admin._apps:
    cred = credentials.Certificate(CRED_PATH)
    firebase_admin.initialize_app(cred, {'databaseURL': DB_URL})

# Setup Open-Meteo
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
client = openmeteo_requests.Client(session=retry_session)

In [9]:
# ================= 🎛️ SETTING BIAS MANUAL =================
# Masukkan angka (positif untuk nambah, negatif untuk kurang)
# Contoh: BIAS_TEMP = 2.0 (Suhu ditambah 2 derajat)
BIAS_TEMP = 2.0      # Bias Suhu (°C)
BIAS_HUM  = 7.0      # Bias Kelembapan (%)
BIAS_DEW  = 4.0      # Bias Titik Embun (°C)
BIAS_PRES = 4.0      # Bias Tekanan (hPa)
BIAS_RAIN = 0.0      # Bias Hujan (mm) - Hati-hati, hujan gak boleh minus!

def get_data_interpolated():
    url = "https://archive-api.open-meteo.com/v1/archive" 
    
    params = {
        "latitude": LAT, "longitude": LON,
        "start_date": TARGET_DATE, "end_date": TARGET_DATE,
        "hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", "surface_pressure", "rain"],
        "timezone": "Asia/Jakarta"
    }
    
    print("🚀 Mengambil data Open-Meteo...")
    try:
        responses = client.weather_api(url, params=params)
        response = responses[0]
    except Exception as e:
        print(f"❌ Error API: {e}")
        return None

    hourly = response.Hourly()
    df = pd.DataFrame({
        "date": pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left"
        ),
        "temperature": hourly.Variables(0).ValuesAsNumpy(),
        "humidity": hourly.Variables(1).ValuesAsNumpy(),
        "dew": hourly.Variables(2).ValuesAsNumpy(),
        "pressure": hourly.Variables(3).ValuesAsNumpy(),
        "rain": hourly.Variables(4).ValuesAsNumpy()
    })
    
    # 1. Konversi ke WIB
    df = df.set_index('date')
    df.index = df.index.tz_convert('Asia/Jakarta')
    
    # 2. Resample ke Menit & Interpolasi
    df = df.resample('1T').asfreq()
    df = df.interpolate(method='time')
    
    # 3. Potong Waktu
    start = f"{TARGET_DATE} {START_TIME_WIB}"
    end   = f"{TARGET_DATE} {END_TIME_WIB}"
    df = df.loc[start:end]
    
    # 4. Cleaning
    df['rain'] = df['rain'].fillna(0) 
    df = df.dropna()
    
    return df

def upload_fix(df):
    ref = db.reference(DB_NODE_PATH)
    print(f"🔥 Memulai Upload ({len(df)} data) dengan Bias...")
    print(f"   🔧 Bias: Temp={BIAS_TEMP}, Hum={BIAS_HUM}, Pres={BIAS_PRES}")
    
    for index, row in df.iterrows():
        # Key Unix Timestamp
        unix_timestamp = int(index.timestamp())
        key = str(unix_timestamp) 
        
        # --- APLIKASI BIAS DI SINI ---
        val_temp = row['temperature'] + BIAS_TEMP
        val_hum  = row['humidity'] + BIAS_HUM
        val_dew  = row['dew'] + BIAS_DEW
        val_pres = row['pressure'] + BIAS_PRES
        val_rain = row['rain'] + BIAS_RAIN
        
        # Jaga-jaga biar hujan & kelembapan gak minus (Physics check)
        if val_rain < 0: val_rain = 0
        if val_hum < 0: val_hum = 0
        if val_hum > 100: val_hum = 100

        data = {
            'temperature': round(float(val_temp), 2),
            'humidity': round(float(val_hum), 2),
            'dew': round(float(val_dew), 2),
            'pressure': round(float(val_pres), 2),
            'rainrate': round(float(val_rain), 2),
            'rainfall': round(float(val_rain), 2),
            
            'timestamp': unix_timestamp,
        }
        
        try:
            ref.child(key).update(data)
            if unix_timestamp % 600 == 0: 
                print(f"   ✅ Key: {key} -> T:{data['temperature']} H:{data['humidity']}")
        except Exception as e:
            print(f"❌ Gagal pada {key}: {e}")
            
    print("✨ Selesai Upload!")

if __name__ == "__main__":
    # --- INPUT MANUAL BIAS (Opsional, kalau mau ditanya saat run) ---
    print("\n--- KONFIGURASI BIAS ---")
    try:
        in_temp = input(f"Masukkan Bias SUHU (Default {BIAS_TEMP}): ")
        if in_temp: BIAS_TEMP = float(in_temp)
        
        in_hum = input(f"Masukkan Bias HUMIDITY (Default {BIAS_HUM}): ")
        if in_hum: BIAS_HUM = float(in_hum)
        
        in_pres = input(f"Masukkan Bias TEKANAN (Default {BIAS_PRES}): ")
        if in_pres: BIAS_PRES = float(in_pres)
        
    except ValueError:
        print("❌ Input salah! Menggunakan default 0.")

    # Eksekusi
    df_hasil = get_data_interpolated()
    
    if df_hasil is not None and not df_hasil.empty:
        print("\n" + "="*40)
        print("👀 PREVIEW (Data + Bias)")
        print("="*40)
        
        # Bikin preview yang sudah kena bias
        preview_df = df_hasil.copy()
        preview_df['temperature'] += BIAS_TEMP
        preview_df['humidity'] += BIAS_HUM
        preview_df['pressure'] += BIAS_PRES
        
        sampel_waktu = df_hasil.index[0]
        sampel_key = str(int(sampel_waktu.timestamp()))
        
        print(preview_df[['temperature', 'humidity', 'pressure']].head())
        print(f"\nContoh Key: {sampel_key}")
        
        x = input("\nGas upload? (y/n): ")
        if x.lower() == 'y':
            upload_fix(df_hasil)
        else:
            print("Batal.")
    else:
        print("Data kosong.")


--- KONFIGURASI BIAS ---
🚀 Mengambil data Open-Meteo...

👀 PREVIEW (Data + Bias)
                           temperature   humidity     pressure
date                                                          
2025-11-29 03:10:00+07:00    24.502001  94.933418  1010.223877
2025-11-29 03:11:00+07:00    24.494501  94.932777  1010.230408
2025-11-29 03:12:00+07:00    24.487001  94.932137  1010.236938
2025-11-29 03:13:00+07:00    24.479502  94.931488  1010.243469
2025-11-29 03:14:00+07:00    24.472002  94.930847  1010.250061

Contoh Key: 1764360600
🔥 Memulai Upload (277 data) dengan Bias...
   🔧 Bias: Temp=2.0, Hum=7.0, Pres=4.0
   ✅ Key: 1764360600 -> T:24.5 H:94.93
   ✅ Key: 1764361200 -> T:24.43 H:94.93
   ✅ Key: 1764361800 -> T:24.35 H:94.92
   ✅ Key: 1764362400 -> T:24.28 H:94.91
   ✅ Key: 1764363000 -> T:24.2 H:94.91
   ✅ Key: 1764363600 -> T:24.13 H:94.9
   ✅ Key: 1764364200 -> T:24.08 H:94.76
   ✅ Key: 1764364800 -> T:24.03 H:94.62
   ✅ Key: 1764365400 -> T:23.98 H:94.48
   ✅ Key: 1764

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=200afad5-8991-4e20-8a55-8751c7aff3b5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>